In [2]:
!nvidia-smi

Wed Nov  8 16:16:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 395 (delta 92), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (395/395), 108.50 KiB | 5.17 MiB/s, done.
Resolving deltas: 100% (194/194), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import numpy as np
import cudf
from cuml import train_test_split
from cuml.preprocessing import StandardScaler
from cuml.ensemble import RandomForestRegressor
from cuml.metrics import mean_squared_error

In [3]:
df_reg = cudf.read_csv('/content/regression_processed.csv')
df_reg.head()

,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,...,area_b,area_c,area_d,product_a,product_b,product_c,product_d,product_e,product_f,product_g
0,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,...,0,1,0,0,1,0,0,0,0,0
1,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,...,0,1,0,0,0,1,0,0,0,0
2,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,...,0,1,0,0,1,0,0,0,0,0
3,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,...,0,1,0,0,1,0,0,0,0,0
4,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,...,0,0,1,1,0,0,0,0,0,0


In [5]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_reg.drop(['age'], axis=1))

In [8]:
x = df_scaled.astype("float32")
y = df_reg['age'].astype("float32")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=500, random_state=42, n_streams=1)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

score = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Final score (RMSE): {score}")

Final score (RMSE): 0.531491756439209
